In [0]:
df = spark.read.table("bikes.bronze.erp_loc_a101")

Importing the libraries

In [0]:
from pyspark.sql.functions import col, trim, when, upper,StringType, initcap

triming the columns

In [0]:
for field in df.schema:
  if field.dataType == StringType():
    df = df.withColumn(field.name, trim(col(field.name)))

Clean the values and rename columns names

In [0]:
# clean fun
df_clean = (
    df

    # Rename columns
    .withColumnRenamed("CID", "customer_id")
    .withColumnRenamed("CNTRY", "country")
    
    # Standardize country names
    .withColumn(
        "country",
        when(col("country").isin("US", "USA", "UNITED STATES"), "United States")
        .when(col("country").isin("DE", "GERMANY"), "Germany")
        .when(col("country").isin("UNITED KINGDOM", "UK"), "United Kingdom")
        .when(col("country") == "AUSTRALIA", "Australia")
        .when(col("country") == "CANADA", "Canada")
        .when(col("country") == "FRANCE", "France")
        .when(col("country").isNull() | (col("country") == ""), "Unknown")
        .otherwise(initcap(col("country")))
    )
)

df = df_clean


Save the table

In [0]:
df.write.mode("overwrite").saveAsTable("bikes.silver.erp_loc")